In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/dataset_LogETL_LogServer.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 97 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Date et heure                                    1249 non-null   object 
 1   Column Name Error                                1249 non-null   float64
 2   Connection Timeout Error                         1249 non-null   float64
 3   Create TABLE Error                               1249 non-null   float64
 4   Date Format Error                                1249 non-null   float64
 5   Field Size Error                                 1249 non-null   float64
 6   Guidez Atelier Error                             1249 non-null   float64
 7   Index Error (similar to unique index error)      1249 non-null   float64
 8   Object Reference Error                           1249 non-null   float64
 9   Other Error                   

In [9]:
df['Date et heure'] = pd.to_datetime(df['Date et heure'])

df['Heure'] = df['Date et heure'].dt.hour

In [10]:
df['Jour semaine'] = df['Date et heure'].dt.day_of_week

In [11]:
df[['Date et heure', 'Heure', 'Jour semaine']]

,Date et heure,Heure,Jour semaine
0,2024-08-24 23:00:00,23,5
1,2024-08-25 00:00:00,0,6
2,2024-08-25 01:00:00,1,6
3,2024-08-25 02:00:00,2,6
4,2024-08-25 03:00:00,3,6
...,...,...,...
1244,2024-10-15 19:00:00,19,1
1245,2024-10-15 20:00:00,20,1
1246,2024-10-15 21:00:00,21,1
1247,2024-10-15 22:00:00,22,1


In [12]:
df[df['Total_Errors'] > 5000]

,Date et heure,Column Name Error,Connection Timeout Error,Create TABLE Error,Date Format Error,Field Size Error,Guidez Atelier Error,Index Error (similar to unique index error),Object Reference Error,Other Error,...,Processeur 3(%),Processeur 4(%),Processeur 5(%),Processeur 6(%),Processeur 7(%),Processeur 8(%),Temps mort cpu(%),Couverture cpu(%),Heure,Jour semaine
108,2024-08-29 11:00:00,0.0,0.0,0.0,0.0,197560.0,0.0,0.0,0.0,0.0,...,12.0833,12.8333,11.0,6.6667,11.75,8.5833,0.0,4000.0,11,3


In [13]:
df_filtre = df[(df['Jour semaine'] == 3) & (df['Heure'] == 11) & (df['Date et heure'] != "2024-08-29 11:00:00")]

In [15]:
ligne_exclue = df[df['Date et heure'] == "2024-08-29 11:00:00"][df.columns[39:97]]

In [16]:
# Calcul de la moyenne par colonne (colonnes 39 à 96)
moyenne_par_colonne = df_filtre.iloc[:, 39:97].mean()

In [18]:
ligne_exclue

,Somme (Volume)(Mo),Somme (Débit)(Mbit/s),Trafic entrant (Volume)(o),Trafic entrant (Débit)(o/s),Trafic sortant (Volume)(o),Trafic sortant (Débit)(o/s),Paquets (Volume)(o),Paquets (Débit)(o/s),Paquets reçus (Volume)(o),Paquets reçus (Débit)(o/s),...,Processeur 1(%),Processeur 2(%),Processeur 3(%),Processeur 4(%),Processeur 5(%),Processeur 6(%),Processeur 7(%),Processeur 8(%),Temps mort cpu(%),Couverture cpu(%)
108,6.511839e+09,1808884.481,5.963285e+09,1656504.795,548554680.0,152379.6862,5278226.489,1466.2066,4501933.114,1250.5648,...,6.5833,10.9167,12.0833,12.8333,11.0,6.6667,11.75,8.5833,0.0,4000.0


In [17]:
moyenne_par_colonne

Somme (Volume)(Mo)                                 3.794075e+09
Somme (Débit)(Mbit/s)                              1.053929e+06
Trafic entrant (Volume)(o)                         3.347586e+09
Trafic entrant (Débit)(o/s)                        9.299019e+05
Trafic sortant (Volume)(o)                         4.464889e+08
Trafic sortant (Débit)(o/s)                        1.240269e+05
Paquets (Volume)(o)                                3.008426e+06
Paquets (Débit)(o/s)                               8.356891e+02
Paquets reçus (Volume)(o)                          2.556066e+06
Paquets reçus (Débit)(o/s)                         7.100311e+02
Paquets envoyés (Volume)(o)                        4.523609e+05
Paquets envoyés (Débit)(o/s)                       1.256581e+02
Monodiffusion entrante (Volume)(o)                 2.544111e+06
Monodiffusion entrante (Débit)(o/s)                7.067104e+02
Monodiffusion sortante (Volume)(o)                 4.523535e+05
Monodiffusion sortante (Débit)(o/s)     

In [27]:
df_resultat = pd.DataFrame({
    "Moyenne": moyenne_par_colonne,
    "Valeur_Ligne_Exclue": ligne_exclue.iloc[0] if not ligne_exclue.empty else None
})


In [33]:
df_resultat['Différence (b - a)'] = df_resultat['Valeur_Ligne_Exclue'] - df_resultat['Moyenne']

In [48]:
df_resultat['Différence %'] = (df_resultat['Valeur_Ligne_Exclue'] - df_resultat['Moyenne']) / df_resultat['Moyenne'] * 100

In [49]:
df_resultat["Différence %"] = df_resultat["Différence %"].apply(lambda x: f"{x:.2f}%")

In [50]:
df_resultat

,Moyenne,Valeur_Ligne_Exclue,Différence (b - a),Différence %
Somme (Volume)(Mo),3.794075e+09,6.511839e+09,2.717764e+09,71.63%
Somme (Débit)(Mbit/s),1.053929e+06,1.808884e+06,7.549557e+05,71.63%
Trafic entrant (Volume)(o),3.347586e+09,5.963285e+09,2.615699e+09,78.14%
Trafic entrant (Débit)(o/s),9.299019e+05,1.656505e+06,7.266029e+05,78.14%
Trafic sortant (Volume)(o),4.464889e+08,5.485547e+08,1.020658e+08,22.86%
Trafic sortant (Débit)(o/s),1.240269e+05,1.523797e+05,2.835283e+04,22.86%
Paquets (Volume)(o),3.008426e+06,5.278226e+06,2.269800e+06,75.45%
Paquets (Débit)(o/s),8.356891e+02,1.466207e+03,6.305175e+02,75.45%
Paquets reçus (Volume)(o),2.556066e+06,4.501933e+06,1.945868e+06,76.13%
Paquets reçus (Débit)(o/s),7.100311e+02,1.250565e+03,5.405337e+02,76.13%
